Imports

In [49]:
import pandas as pd
import statsmodels.api as sm
import numpy as np
import os
from matplotlib import pyplot as plt
from pandas import DataFrame as df

Functions

In [50]:
# Function to plot regression results
def plot_regression_results(results, df, folder_path):
    plots_folder_path = os.path.join(folder_path, 'plots')


Main Execution

In [51]:
# Directories configuration
data_dir = r'C:\Users\{username}\UPDATE THIS PATH TO MATCH YOURS\COVID Sector Analysis PROJECT\Sector_Analysis_Data\Data'
results_dir = r'C:\Users\{username}\UPDATE THIS PATH TO MATCH YOURS\COVID Sector Analysis PROJECT\results'
file_path = os.path.join(data_dir, "Project_Data.csv")
figures_folder = os.path.join(results_dir, "Figures")

# Ensure the results and figures folders exist
os.makedirs(results_dir, exist_ok=True)
os.makedirs(figures_folder, exist_ok=True)

# Load the CSV file into a DataFrame
df = pd.read_csv(file_path)

# Lists and dictionaries setup
portfolios = ['Information Technology', 'Real Estate', 'Materials', 'Health Care', 'Financials',
              'Consumer Discretionary', 'Communication Services', 'Industrials', 'Consumer Staples',
              'Energy', 'Utilities']

independent_var_names = ['Days Since COVID', 'Unemployment Rate']

# Initialize results dictionary
results = {}

# Perform all regressions including the specified third regression first
third_regression_vars = {
    'dependent_var': 'Unemployment Rate',
    'independent_var': 'Days Since COVID'
}
X = sm.add_constant(df[third_regression_vars['independent_var']])
y = df[third_regression_vars['dependent_var']]
model_third = sm.OLS(y, X).fit(cov_type='HC3')
results[(third_regression_vars['dependent_var'], third_regression_vars['independent_var'])] = {'model': model_third}

# Continue with other regressions as previously defined
for portfolio in portfolios:
    for independent_var_name in independent_var_names:
        X = sm.add_constant(df[independent_var_name])
        y = df[portfolio]
        model = sm.OLS(y, X).fit(cov_type='HC3')
        results[(portfolio, independent_var_name)] = {'model': model}

# Save the results and summaries to the text file
results_summary_file = os.path.join(results_dir, 'results_summaries.txt')

with open(results_summary_file, 'w') as f:
    for (dep_var, ind_var), detail in results.items():
        model = detail['model']
        f.write(f"Summary for model with Dependent: {dep_var} and Independent: {ind_var} variables:\n")
        f.write(model.summary().as_text())
        f.write("\n\n")

# Plot and save figures for all regressions
for (dep_var, ind_var), detail in results.items():
    plt.figure(figsize=(8, 5))
    model = detail['model']
    X = df[ind_var] if dep_var != 'Unemployment Rate' else sm.add_constant(df[ind_var])  # Adjust for the third regression
    y_actual = df[dep_var]
    plt.scatter(X[ind_var] if dep_var == 'Unemployment Rate' else X, y_actual, alpha=0.5, label="Actual Data")

    if dep_var == 'Unemployment Rate':  # Adjust for the third regression
        sorted_indices = X[ind_var].argsort()
    else:
        sorted_indices = X.argsort()
    
    X_sorted = X.iloc[sorted_indices]
    y_fitted_sorted = model.fittedvalues.iloc[sorted_indices]
    plt.plot(X_sorted[ind_var] if dep_var == 'Unemployment Rate' else X_sorted, y_fitted_sorted, 'r-', alpha=0.7, label="Fitted Line")
    plt.xlabel(ind_var)
    plt.ylabel(dep_var)
    plt.title(f'Regression Analysis: {dep_var} vs {ind_var}')
    plt.legend()

    # Saving the plot
    plot_filename = f'{dep_var.replace(" ", "_")}_VS_{ind_var.replace(" ", "_")}.png'
    plot_path = os.path.join(figures_folder, plot_filename)
    plt.savefig(plot_path)
    plt.close()

print(f"The results and summaries have been saved to: {results_summary_file}")
print(f"The regression plots have been saved in: {figures_folder}")

The results and summaries have been saved to: C:\Users\caliB\Downloads\Python Projects\COVID PROJECT\results\results_summaries.txt
The regression plots have been saved in: C:\Users\caliB\Downloads\Python Projects\COVID PROJECT\results\Figures
